In [2]:
import numpy as np
from header import *
from nsp.utils.base_conv import *
import argparse
import sys
sys.path.append('../nsp')
from nsp.utils.func import *
from nsp.utils.local2global import *

In [105]:
E4 = np.load("/home/user/project/doc/data/shastry_exact_J1=1.npy")

In [109]:
T = 1
beta = 1/T
np.sum(E4 * np.exp(-beta*E4))/np.sum(np.exp(-beta*E4))

-5.585519722674071

## prepare local ham

In [110]:
u = np.array([
    [0,1,0,0],
    [1/np.sqrt(2), 0, 1/np.sqrt(2), 0],
    [-1/np.sqrt(2), 0, 1/np.sqrt(2), 0],
    [0,0,0,1]
])

U = np.kron(u, u)

In [122]:
J = [0.35, 1] #J, J_D

In [123]:
Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2

SzSz = np.kron(Sz,Sz).real.astype(np.float64)
SxSx = np.kron(Sx,Sx).real.astype(np.float64)
SySy = np.kron(Sy,Sy).real.astype(np.float64)

lh = SzSz + SxSx + SySy
lh = -lh

In [124]:
H1 = sum_ham(J[0]*lh, [[1,2],[1,3]], 4, 2)
H1 += sum_ham(J[1]*lh/4, [[0,1],[2,3]], 4, 2)

H2 = sum_ham(J[0]*lh, [[0,2],[0,3]], 4, 2)
H2 += sum_ham(J[1]*lh/4, [[0,1],[2,3]], 4, 2)

E1 = np.linalg.eigvalsh(H1)
E2 = np.linalg.eigvalsh(H2)

In [125]:
H1, H2 = U.T@H1@U, U.T@H2@U
print(H1)

[[ 3.75000000e-01  0.00000000e+00  6.93889390e-18  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -5.55111512e-18
   6.93889390e-18  0.00000000e+00  6.93889390e-18  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.25000000e-01  0.00000000e+00  0.00000000e+00
  -3.93551224e-18  0.00000000e+00 -1.75000000e-01  0.00000000e+00
   0.00000000e+00  1.75000000e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.54074396e-33  0.00000000e+00  1.25000000e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.75000000e-01
   1.54074396e-33  0.00000000e+00 -1.54074396e-33  0.00000000e+00
   0.00000000e+00  1.75000000e-01  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.25000000e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.75000000e-01
   0.00

In [126]:
E1

array([-0.3      , -0.3      , -0.3      , -0.3      , -0.3      ,
       -0.1628123, -0.1628123, -0.1628123,  0.125    ,  0.125    ,
        0.125    ,  0.225    ,  0.3378123,  0.3378123,  0.3378123,
        0.375    ])

### prepair plaquette 

In [ ]:
square1 = [[0,1], [1,2], [2,3], [3,0]]
square2 = [[4,5], [5,6], [6,7], [7,4]]
H1 = sum_ham(J[0]*lh/4, square1 + square2, 8, 2)
H1 += sum_ham(J[1]*lh, [[1,7]], 4, 2)

H2 = sum_ham(J[0]*lh/4, square1 + square2, 8, 2)
H2 += sum_ham(J[1]*lh, [[2,4]], 4, 2)

In [98]:
os.getcwd()

'/home/user/project/python/exact/SS'

In [100]:
H1 = np.load("../../array/SS/plq_original_J=[1.0, 1]/0.npy")

In [104]:
(2**4)**4

65536

In [102]:
print(H1.shape)

(256, 256)


## optimize unitary

### dimer basis

In [53]:
D = 4

models = nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64)

In [54]:
loss_f = nsp.loss.MES
from datetime import datetime
from random import randint

In [62]:
D = 4
M = 10
loss1 = loss_f(H1, [D, D], pout = False)
loss2 = loss_f(H2, [D, D], pout = False)
best_fun = 1E10

for _ in range(M):
    seed = randint(0, 2<<32 - 1)
    torch.manual_seed(seed)
    np.random.seed(seed)
#     models = [nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64) for _ in range(2)]
    model = nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64)
    cg = RiemanNonTransUnitaryCG([(model, model)]*2, [loss1, loss2], pout = False)
    solver = UnitaryNonTransTs(cg, af=False)
    ret = solver.run(2000, False)
    print(f"res = {ret.fun} / seed = {seed}")
    if ret.fun < best_fun:
        print(f"\nbest_fun updated : {ret.fun}\n")
        best_fun = ret.fun
        best_model = ret.model
# LHs = [loss._transform_kron([best_model[i].matrix(), best_model[(i+1)%L].matrix()], original=True).detach().numpy() for i in range(L)]
# folder = f"../../array/FL/optm_nt{L}Jp={(J[0]):.2}_Jx={(J[1]):.2}_{ham_type}_{loss_name}"
# co = nsp.utils.base_conv.change_order

target loss      : 1.3000000119
initial loss     : 1.7798828929
loss upper bound : 1.5000000119




  1%|██▉                                                                                                                                                                                                                         | 27/2000 [00:00<00:23, 83.77it/s, iter=26, loss=1.30000]


No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
stack in local minimum --> break loop
res = 1.3000000119208908 / seed = 3046045476

best_fun updated : 1.3000000119208908

target loss      : 1.3000000119
initial loss     : 1.9955372456
loss upper bound : 1.5000000119




  2%|████▋                                                                                                                                                                                                                       | 43/2000 [00:00<00:29, 67.35it/s, iter=42, loss=1.50000]


No local minimum found
stack in local minimum --> break loop
res = 1.5000000119206234 / seed = 3222352951
target loss      : 1.3000000119
initial loss     : 1.7897753081
loss upper bound : 1.5000000119




...

No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
stack in local minimum --> break loop
res = 1.3000000119210653 / seed = 2851178001
target loss      : 1.3000000119
initial loss     : 2.0629344310
loss upper bound : 1.5000000119




  3%|██████▌                                                                                                                                                                                                                     | 60/2000 [00:00<00:29, 65.37it/s, iter=59, loss=1.77918]


stack in local minimum --> break loop
res = 1.7791814384438325 / seed = 1859769114
target loss      : 1.3000000119
initial loss     : 1.9053715810
loss upper bound : 1.5000000119




  1%|██▉                                                                                                                                                                                                                         | 27/2000 [00:00<00:21, 90.16it/s, iter=26, loss=1.30000]

No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
stack in local minimum --> break loop
res = 1.3000000119213626 / seed = 2233820119


### optimzie for many J

In [92]:
import re

def process_line(stdout):
    for line in str(stdout).split("\\n"):
        pat = "best fun is : "
        if pat in line:
            line = line.replace(pat,"")
            return float(line)
            


In [87]:
print(f"{0.123:.2}")

0.12


In [127]:
import numpy as np
import os 
os.chdir("/home/user/project/python/exact/SS")
import subprocess
energy_array = []
sign_array = []
time_array = []
for J in np.arange(0.1, 1.1, 0.1):
    out = subprocess.Popen(["python", "make_ss_local.py", "-l", "dimer_basis" ,"-J", f"{J:.1}", "-M", "100"], 
               stdout=subprocess.PIPE, 
               stderr=subprocess.STDOUT)
    stdout,stderr = out.communicate()

    best = process_line(stdout)
    print(f"best fun at J = {J:.2} is : ",best)
#     energy_array.append(energy)
#     sign_array.append(sign)
#     time_array.append(time)

best fun at J = 0.1 is :  None
best fun at J = 0.2 is :  None
best fun at J = 0.3 is :  None
best fun at J = 0.4 is :  None
best fun at J = 0.5 is :  None
best fun at J = 0.6 is :  None
best fun at J = 0.7 is :  None
best fun at J = 0.8 is :  None
best fun at J = 0.9 is :  None
best fun at J = 1.0 is :  None
